In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import os
import folium
import json

In [2]:
path = r'C:\Users\karim\Documents\achievement 6 data sourcing'

In [3]:
rank= pd.read_csv(os.path.join(path, '02 Data', 'prepared Data', 'times_cleaned.csv'), index_col = False)

In [4]:
rank.head()

Unnamed: 0 world_rank                        university_name  \
0           0          1                     Harvard University   
1           1          2     California Institute of Technology   
2           2          3  Massachusetts Institute of Technology   
3           3          4                    Stanford University   
4           4          5                   Princeton University   

                    country  teaching international  research  citations  \
0  United States of America      99.7          72.4      98.7       98.8   
1  United States of America      97.7          54.6      98.0       99.9   
2  United States of America      97.8          82.3      91.4       99.9   
3  United States of America      98.3          29.5      98.1       99.2   
4  United States of America      90.9          70.3      95.4       99.9   

  income total_score  year  
0   34.5        96.1  2011  
1   83.7          96  2011  
2   87.5        95.6  2011  
3   64.3        94.3  2011  
4      -        94.2  2011

In [5]:
rank.shape

(2603, 11)

# Data wrangling and cleaning

In [7]:
with open(r'C:\Users\karim\Downloads\world-countries.json',) as ctr:
    ctr_file = json.load(ctr)  # Parse JSON into a Python dictionary or list

# Access or manipulate the data
print(ctr_file)  # Display the content

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'Afghanistan'}, 'geometry': {'type': 'Polygon', 'coordinates': [[[61.210817, 35.650072], [62.230651, 35.270664], [62.984662, 35.404041], [63.193538, 35.857166], [63.982896, 36.007957], [64.546479, 36.312073], [64.746105, 37.111818], [65.588948, 37.305217], [65.745631, 37.661164], [66.217385, 37.39379], [66.518607, 37.362784], [67.075782, 37.356144], [67.83, 37.144994], [68.135562, 37.023115], [68.859446, 37.344336], [69.196273, 37.151144], [69.518785, 37.608997], [70.116578, 37.588223], [70.270574, 37.735165], [70.376304, 38.138396], [70.806821, 38.486282], [71.348131, 38.258905], [71.239404, 37.953265], [71.541918, 37.905774], [71.448693, 37.065645], [71.844638, 36.738171], [72.193041, 36.948288], [72.63689, 37.047558], [73.260056, 37.495257], [73.948696, 37.421566], [74.980002, 37.41999], [75.158028, 37.133031], [74.575893, 37.020841], [74.067552, 36.836176], [72.920025, 36.720007], [71.846292, 36.5

In [8]:
rank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2603 entries, 0 to 2602
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       2603 non-null   int64  
 1   world_rank       2603 non-null   object 
 2   university_name  2603 non-null   object 
 3   country          2603 non-null   object 
 4   teaching         2603 non-null   float64
 5   international    2603 non-null   object 
 6   research         2603 non-null   float64
 7   citations        2603 non-null   float64
 8   income           2603 non-null   object 
 9   total_score      2603 non-null   object 
 10  year             2603 non-null   int64  
dtypes: float64(3), int64(2), object(6)
memory usage: 223.8+ KB


In [9]:
rank['world_rank'] = pd.to_numeric(rank['world_rank'], errors='coerce')

In [10]:
# Check for null values
rank.isnull().sum()

Unnamed: 0            0
world_rank         1402
university_name       0
country               0
teaching              0
international         0
research              0
citations             0
income                0
total_score           0
year                  0
dtype: int64

In [11]:
duprank = rank[rank.duplicated()]

In [12]:
duprank.head()

Empty DataFrame
Columns: [Unnamed: 0, world_rank, university_name, country, teaching, international, research, citations, income, total_score, year]
Index: []

no duplicates

# 04 Create Choropleth

In [30]:
data_to_plot1=rank[['country', 'teaching']]

In [32]:
data_to_plot1.head()

country  teaching
0  United States of America      99.7
1  United States of America      97.7
2  United States of America      97.8
3  United States of America      98.3
4  United States of America      90.9

In [34]:
data_to_plot1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2603 entries, 0 to 2602
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   country   2603 non-null   object 
 1   teaching  2603 non-null   float64
dtypes: float64(1), object(1)
memory usage: 40.8+ KB


In [52]:
data_to_plot1['countries_teaching'] = data_to_plot1.groupby('country')['teaching'].transform('mean')


C:\Users\karim\AppData\Local\Temp\ipykernel_2888\926059508.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_to_plot1['countries_teaching'] = data_to_plot1.groupby('country')['teaching'].transform('mean')


In [54]:
data_to_plot1.head()

country  teaching  countries_teaching
0  United States of America      99.7           47.920152
1  United States of America      97.7           47.920152
2  United States of America      97.8           47.920152
3  United States of America      98.3           47.920152
4  United States of America      90.9           47.920152

In [56]:

map = folium.Map(location = [52.5200, 13.4050], zoom_start=2)


folium.Choropleth(
    geo_data = ctr_file, 
    data = data_to_plot1,
    columns = ['country','countries_teaching'],
    key_on = 'feature.properties.name', 
    fill_color = 'YlOrBr', fill_opacity=0.8, line_opacity=0.8,
    legend_name = "best teaching indicator").add_to(map)
folium.LayerControl().add_to(map)

map

# Observations
# i chose to analyse the teaching indicator because it has a big influence on the universities ranking. 

# the choropleth map results contributed to realise  that united states  have the best universities ranking,this can lead to the hypothesis i wrote earlier 'the sames universities and countries are maintaining the same ranking places'.